In [ ]:
path_to_data = './data/'

In [ ]:
import pandas as pd
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt

Loading our data

In [ ]:
def loading_data(path, f_name):
    
    X_data=[]
    y_data=[]
    data_path = os.path.join(path, '{}.csv'.format(f_name))
    data = pd.read_csv(data_path)
    
    X_id = data['id_image'].values
    Y = data['label_image'].values
    
    for file in X_id:
        img = cv2.imread(os.path.join(path, "images", str(file)))
        X_data.append(np.asarray(img))                    
    
    return (np.array(X_data),np.array(Y))

In [ ]:
X_data, y_data = loading_data(path_to_data, 'train')

Verify the shape of the date to ensure that it is loaded

In [ ]:
X_data.shape

In [ ]:
distributions = {}
for new_data_point in y_data :
    
    if new_data_point in distributions:
        distributions[new_data_point] += 1
    else:
        distributions[new_data_point] = 1
    

plt.bar(distributions.keys(), distributions.values());

plt.ylabel('number of images ')
plt.title('classes')
 
plt.show()

We notice that the 3 classes "city", "forest" and "non dechet" have more data than the classes "medical waste" and "domestique". So we should consider the fact the dataset is unbalanced in our models.

We show here some images (one image for each classes)

In [ ]:
plt.figure(figsize=(40, 40))
L=[3,144,282,351,500]
titles=list(np.unique(y_data))
for i in range(5):
    ax=plt.subplot(1,5,i+1)
    plt.imshow(X_data[L[i]])
    plt.title(titles[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show

For example, we notice that even the photo of "non dechet" class is taken in a forest, so the model must be able to detect the place where the photo is taken, if there is trash or not and sometimes the type of the trash (like the case of medical waste class)

Read the csv file.
The csv file containes the id and the labels of each image

In [ ]:
pd.read_csv("data/train.csv").head()

## Data Preprocessing

The labels of classes are strings, so we change our labels to numerical labels using dicto_generator function.

In [ ]:
def dictio_generator(s):
    d = dict()
    id_dictionnaire = 0
    d[s[0]] = id_dictionnaire
     
    for c in s : 
        if c not in d : 
            id_dictionnaire+= 1
            d[c] = id_dictionnaire
    return d

In [ ]:
# transforming  categorical Labels into numeric ones 
dicta = dictio_generator(np.unique(y_data))
y_data_numeric = np.array([ dicta[i] for i in y_data ])

We split the data on train and test sets to test our model manually. However, we will not need this split to run the model on the ramp

In [ ]:
from sklearn.model_selection import train_test_split
import keras

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_numeric, test_size=0.33, random_state=42)
print ('x_train size: ' , X_train.shape)
print ('x_test size:  ' , X_test.shape)
print ('y_train size: ' , y_train.shape)
print ('y_test size:  ' , y_test.shape)

We have 6 classes. Thus, we need to use One hot encoding method to have a better representation of our targets 

In [ ]:
# normalizing
X_train = X_train.astype('float32') / 255.
X_test  = X_test.astype('float32') / 255.
one_hot_encoded_y_train = keras.utils.to_categorical(y_train,len(dicta))
one_hot_encoded_y_test = keras.utils.to_categorical(y_test,len(dicta))

## Deep Learning Model 

When managing images, data with high level complexity, we thought of Deep Neural networks as models capable of extracting features in an automatic way.

for that, using keras package we developed  a convolutional neural net with two hidden layer and a fully connected part that outputs a vector with dimension (num_classes,1)


Model is as such: 

   ![Alt text](https://jhui.github.io/assets/cnn/convolution_b1.png)
   
Where each layer is extracting more relative features from previous representation of data

First, we upload the necessary packages which are more likely to be used to develop the neural net

In [ ]:
from __future__ import print_function
import numpy as np

import keras
from keras.datasets import mnist

from keras.models import Sequential 
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D 
from keras.optimizers import Adam, SGD, RMSprop

from keras.utils import np_utils 
from keras.callbacks import TensorBoard 
from keras import regularizers 
from time import time 

import matplotlib.pyplot as plt

Then we fix the hyper-parameters 

In [ ]:
np.random.seed(1671) # for reproducibility
# network and training

NB_EPOCH = 10 # number of epochs/episodes 
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD() # optimizer, explained later in this chapter
N_HIDDEN = 128

VALIDATION_SPLIT=0.1 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3
num_classes = 5 # number of outputs = number of digits

We have implementing a simple neural network define by:
![Alt text](./graph/architecture.png")
Model is defined as such 

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation=None, input_shape=(128,128,3)))
model.add(Conv2D(32,  (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(DROPOUT))

model.add( Conv2D (64,(3,3), activation = 'relu'))
model.add( Conv2D (64,(3,3), activation = 'relu'))
model.add( MaxPooling2D ( pool_size =(2,2)))
model.add(Dropout(DROPOUT))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
          
        
sgd = SGD(lr=0.01, decay=0, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
        
# training and fitting the model 

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs_CNN', histogram_freq=0, 
                            batch_size=32, 
                            write_graph=True, 
                            write_grads=True, write_images=True,
                            embeddings_freq=0, embeddings_layer_names=None,
                            embeddings_metadata=None)

""" tensorboard --logdir ./logs_CNN"""

Before doing any training, we need to resize the images in the goal that they matches the input shape of the model.

In [ ]:
X_train_resize = np.zeros(( len(X_train), 128, 128, 3))
for i in range(len(X_train)):
    X_train_resize[i] = cv2.resize(X_train[i], (128, 128))
    
X_test_resize = np.zeros(( len(X_test), 128, 128, 3))
for i in range(len(X_test)):
    X_test_resize[i] = cv2.resize(X_test[i], (128, 128))

We train our neural network for a fixed number of epochs and with the respect to the other hyper-parameters 

In [ ]:
history = model.fit(X_train_resize, one_hot_encoded_y_train, batch_size=BATCH_SIZE,
                   epochs=NB_EPOCH,validation_split = VALIDATION_SPLIT,
          callbacks=[tensorboard])


We can then move on analysing the performance of the model 

In [ ]:
# Metrics evaluation 
#tensorboard --logdir=logs/
score = model.evaluate(X_test_resize, one_hot_encoded_y_test, verbose=VERBOSE,batch_size=BATCH_SIZE)
print("Test score:", score[0])
print('Test accuracy:', score[1])


Besides, an important point to verify is to check if we are learning something and not overfitting and learning too much about our train data 

In [ ]:
# tensorboard logs 
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Hint : if the gap between the train plot and the validation one is too large, then we are facing an overfitting problem 
we might want to adjust our hyper-parameters, or manage our data set size...

## Second variant of CNN: CNN + Data augmentation

Having a relatively small data set, one way to adress the overfitting problem is to use the data augmentation method,
meaning apply basic modifications on input images such as ( flip, zoom, random noise, rotation...).

We then conserve the information within our input data and enlarge our data set, which has strong impact on our metrics results and the model performance.

One might use the keras ImageDataGeneraor and choose the type of modifications to apply

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In chase of computation limitations, we might generate data while performing the training of the network, in a way to not lose much time and space to create new data points(images) and save them in independant folders which might take huge space

We maintain the same structure of the model, but this time we fit our model with data generated from data augmentation process, hyper-parameters could be adjusted also as needed 

In [ ]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_resize)

# fits the model on batches with real-time data augmentation:
history_augmented = model.fit_generator(datagen.flow(X_train_resize, one_hot_encoded_y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(X_train_resize) / 32, epochs=NB_EPOCH,validation_steps=2)

After that we porceed to evaluate our model efficiency, when evaluating the model, we should maintain the same transformations applied to our train data as well for our test data 

In [ ]:
test_datagen= datagen.flow(X_test_resize, one_hot_encoded_y_test, batch_size=BATCH_SIZE)

In [ ]:
scoreSeg = model.evaluate_generator(test_datagen, 100)
print("loss_function",scoreSeg[0], "Accuracy = ",scoreSeg[1])

## Submission files

After you find a good model, you have to write an ImageClassifier class on the goal to do a submission.<br/>
Here, is a example of ImageClassifier class implementing the model explain above.


In [ ]:
# %load ./submissions/starting_kit/image_classifier.py
import numpy as np


from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import cv2

class ImageClassifier(object):

    def __init__(self, batch_size=32, epochs=1, validation_split=0.2, dropout=0.3, input_shape_image=(128, 128)):
        
        
        self.batch_size = batch_size
        self.epochs = epochs
        self.step_per_epoch = 20
        self.input_image_shape = input_shape_image
        self.input_shape = self.input_image_shape + (3,)
        self.dropout = dropout
        self.validation_split = validation_split
        self.model = self._build_model()

    def _transform(self, x):   
        x = x / 255.
        x_resize = cv2.resize(x, self.input_image_shape)
        return x_resize
    
    def data_augmentation(self, X):
        datagen = ImageDataGenerator(
                featurewise_center=True,
                featurewise_std_normalization=True,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True)
        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        
        datagen.fit(X)
        return datagen

    def fit(self, img_loader):
        nb = len(img_loader)
        X = np.zeros((nb,) + self.input_shape)
        Y = np.zeros((nb, 5))
        
        for i in range(nb):
            x, y = img_loader.load(i)
            X[i] = self._transform(x)
            Y[i, y] = 1
            
        datagen = self.data_augmentation(X)
        
        self.model.fit_generator(datagen.flow(X, Y, batch_size=self.batch_size), steps_per_epoch=self.step_per_epoch, epochs=self.epochs)
     
    def predict_proba(self, img_loader):
        nb = len(img_loader)
        X = np.zeros((nb, 128, 128, 3))
        for i in range(nb):
            X[i] = self._transform(img_loader.load(i))
        return self.model.predict(X)
    
    def _build_model(self):
        
        model = Sequential()


        model.add(Conv2D(32, (3,3), activation=None, input_shape=self.input_shape))
        model.add(Conv2D(32,  (3,3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(self.dropout))

        model.add( Conv2D (64,(3,3), activation = 'relu'))
        model.add( Conv2D (64,(3,3), activation = 'relu'))
        model.add( MaxPooling2D ( pool_size =(2,2)))
        model.add(Dropout(self.dropout))
        model.add(Flatten())
        model.add(Dense(256,activation = 'relu'))
        model.add(Dropout(0.5))
        model.add(Dense(5, activation='softmax'))
          
        
        sgd = SGD(lr=0.01, decay=0, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
        
        return model


## Local Testing Before Submitted to Ramp

You have to test your submissions files before doing your submissions. In that goal, use ramp_test_submission command. Notice that this unit test run in the folder [`submissions/starting_kit`](/tree/submissions/starting_kit).
Before running the test, make sure you have done the following tasks:

- install ramp-workflow
- write the python file image_classifier.py and put it in the following folder [`submissions/starting_kit`].
- download the data by excuting `python download_data.py`

You have two possibility to test your submissions, a complete test (train and test the model with cross validtion like the server or a quick test (just to make sure the submissions code are working). If the test run, print train and test erros on each fold of the cross validation you can then submit to ramp.

Execute the following cell to do a quick test:

In [ ]:
!ramp_test_submission --quick-test

And the following cell to do a complete test

In [1]:
!ramp_test_submission

Testing Trash classification
Reading train and test files from ./data ...
Reading cv ...
Training .\submissions\starting_kit ...
CV fold 0
Epoch 1/1

20/20 [==============================] - 7s 353ms/step - loss: 1.5855 - acc: 0.3113
	score    acc
	train  0.158
	valid  0.173
	test   0.160
----------------------------
Mean CV scores
----------------------------
	score          acc
	train  0.158 ± 0.0
	valid  0.173 ± 0.0
	test    0.16 ± 0.0
----------------------------
Bagged scores
----------------------------
	score    acc
	valid  0.173
	test   0.160


2019-01-27 23:28:52.336430: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
2019-01-27 23:28:52.929231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: GeForce GTX TITAN X major: 5 minor: 2 memoryClockRate(GHz): 1.076
pciBusID: 0000:03:00.0
totalMemory: 12.00GiB freeMemory: 11.59GiB
2019-01-27 23:28:52.929231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2019-01-27 23:28:53.537632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-01-27 23:28:53.537632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2019-01-27 23:28:53.537632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2019-01-27 23:28:53.553232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1103] Created TensorFlow device (/job:localh